In [83]:
import pandas as pd
import numpy as np
from lxml.html import parse
from urllib2 import urlopen
import re

In [87]:
#function for parsing
def read_rb_stats(season, page=0):
    #read in RB stats for given season from fftoday.com
    rb_column_names = ['Name', 'Team', 'Games', 'RunAtt', 'RunYards', 'RunTD', 'Targets', 'Rec', 'RecYards', 'RecTD', 'FFP', 'FFPPG']
    url_str = 'http://fftoday.com/stats/playerstats.php?Season=%d&GameWeek=&PosID=20&LeagueID=1&order_by=FFPts&sort_order=DESC&cur_page=%d' % (season, page)
    
    #parse html and find the main data table
    parsed = parse(urlopen(url_str))
    page = parsed.getroot()
    tables = page.findall('.//table')
    #get the rows of the player table
    main_table = tables[10]
    rows = main_table.findall('.//tr')
    
    #dataframe we will return
    rb_df = pd.DataFrame(columns=rb_column_names)
    for i,row in enumerate(rows[2:]): #rows[2] is first player entry
        elements = row.findall('.//td') #find all elements in the row
        values = [val.text_content() for val in elements] #make a list of all the values from the row
        rb_df.loc[i] = values
        
    #clean up entries
    rb_df['Name'] = rb_df['Name'].str.replace('[^a-z]', '',flags=re.IGNORECASE) #remove extra stuff from name cell
    rb_df['RunYards'] = rb_df['RunYards'].str.replace(',', '') #remove thousands commas
    rb_df['RecYards'] = rb_df['RecYards'].str.replace(',', '')
    
    return rb_df

In [90]:
df = read_rb_stats(2014,0)
df.head()

,Name,Team,Games,RunAtt,RunYards,RunTD,Targets,Rec,RecYards,RecTD,FFP,FFPPG
0,DeMarcoMurray,DAL,16,393,1845,13,64,57,416,0,304.1,19.0
1,LeVeonBell,PIT,16,290,1361,8,105,83,854,3,287.5,18.0
2,MarshawnLynch,SEA,16,280,1306,13,48,37,367,4,269.3,16.8
3,MattForte,CHI,16,266,1038,6,130,102,808,4,244.6,15.3
4,ArianFoster,HOU,13,260,1246,8,59,38,327,5,235.3,18.1
